# 課題2(Optional) wikipediaリンクで遊ぼう


## PageRank実装してみる - ランダムサンプリング編

### 準備
データを全部使うとRAMが死んでしまうので，適宜サンプリングしたデータのみを使用してメモリを節約する．

In [1]:
from google.colab import auth, drive, files, output
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("/content/drive/My Drive/STEP2020")

In [0]:
size = 10000 #サンプリングするページ数

In [0]:
#pagesの読み込み
f1 = open('txt/wikipedia/pages.txt')
pages = f1.read()
f1.close()
n_pages = pages.split("\n")
pages = [n_pages[i].split("\t")[1] for i in range(len(n_pages)-1)] #最後に空行が入っていたので．
del n_pages[:]

In [0]:
#randomにindexをサンプリング
import random
indexes = [i for i in range(1483277)]
new_indexes = list(sorted(random.sample(indexes, size))) #ここを変えるとページ数が変わる．
del indexes[:]
indexes = new_indexes

In [0]:
#サンプリングしたページとindexの対応を記録
selected_pages = []
page2num = dict()
old2new = dict()
for i in range(len(indexes)):
    selected_pages.append(pages[indexes[i]])
    page2num[pages[indexes[i]]] = i
    old2new[str(indexes[i])] = i
del pages[:]
del indexes[:]
pages = selected_pages

In [0]:
#リンクの読み込み
f1 = open('txt/wikipedia/links.txt')
links = f1.read()
f1.close()
n_links = links.split("\n")
links = [tuple(map(int, n_links[i].split('\t'))) for i in range(len(n_links)-1)] #最後に空行が入っていたので．
del n_links[:]

In [8]:
len(links)

52973671

In [0]:
#必要なリンク以外捨てる
new_links = []
for i in range(len(links)):
    if (old2new.get(str(links[i][0]))) and (old2new.get(str(links[i][1]))): #inで検索するとO(N)の計算時間なので，辞書検索でO(1)にする．
        new_links.append(tuple([old2new[str(links[i][0])], old2new[str(links[i][1])]]))
del links[:]
links = new_links
#links[:5] #確認用

In [10]:
#少ないなリンク..まあしょうがないか
len(links)

2193

In [11]:
#RAMが死にそう..メモリ増やせないものか？
#頑張ってdelでメモリ開放していく．
#あと．for文はコピー造らないようにiで回してアクセス．
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        2.6G        9.6G        908K        452M         11G
Swap:            0B          0B          0B


### PageRank

In [0]:
import time
import numpy as np
import numpy.linalg as LA
#from scipy.sparse.linalg import eigs

In [13]:
#リンク行列の定義
cpu_start = time.time()
p_matrix = np.zeros((size, size))
e_matrix = np.ones((size, size))
PageRank = 100 * np.ones((size, size))

#繊維確率の計算
for i in range(len(links)):
    p_matrix[links[i][0]][links[i][1]] += 1
for i in range(size):
    if sum(p_matrix[i]) != 0:
        p_matrix[i] = p_matrix[i]/sum(p_matrix[i])
google_matrix = 0.85 * p_matrix + 0.15 * (1/size) * e_matrix
#ここからべき乗法の計算
for i in range(20):
    #print('iteration: ', i)
    PageRank = PageRank * google_matrix
w,v = LA.eig(PageRank) #やっぱり固有値の計算が終わらないな..
#w, v = eigs(PageRank, k=10) #scipyのsparseのやつは固有値全部計算してくれるわけではないらしい..
cpu_end = time.time()

print('実行時間：', cpu_end - cpu_start)
#print('eigen value:', w)

実行時間： 941.1972625255585


In [0]:
w_index = list(reversed(sorted([[w[i], i] for i in range(size)])))

In [15]:
#重要度高そうなページを10個出してみる．
#値を見る感じ，リンク数が少なくてスパース過ぎたためほとんど差が出なかったのかなという印象．
#ほんとに？みたいなの結構ランクインしてるし笑
for i in range(10):
    print(pages[w_index[i][1]], w_index[i][0])

オゴデイ・ハーン国 (3.8773213212642+0j)
欠格条項 (3.8773213212641977+0j)
エトヴィン・フィッシャー (3.877321321264196+0j)
Windows3.1 (3.8773213212641955+0j)
瀬戸内工業地域 (3.8773213212641955+0j)
神林長平 (3.8773213212641946+0j)
日本の地方公共団体_(お) (3.8773213212641946+0j)
はと (3.877321321264194+0j)
トランペッター (3.8773213212641924+0j)
東由利町 (3.877321321264192+0j)


#### GPU ver
AttributeError: module 'cupy.linalg' has no attribute 'eig'となってできなかった．  
行列積と固有値演算をGPUでできるライブラリを知りたい． <- そういえば，固有値の計算はGPUだと厳しいのかな(?)

In [0]:
"""
#GPUを使用する場合
import cupy as cp
import cupy.linalg as CLA
#リンク行列の定義
gpu_start = time.time()
p_matrix = cp.zeros((size, size))
e_matrix = cp.ones((size, size))
PageRank = 100 * cp.ones((size, size))
#繊維確率の計算
for i in range(len(links)):
    p_matrix[links[i][0]][links[i][1]] += 1
for i in range(size):
    if sum(p_matrix[i]) != 0:
        p_matrix[i] = p_matrix[i]/sum(p_matrix[i])
google_matrix = 0.85 * p_matrix + 0.15 * (1/size) * e_matrix
#ここからべき乗法の計算
for i in range(20):
    #print('iteration: ', i)
    PageRank = PageRank * google_matrix
w,v = CLA.eig(PageRank) #やっぱり固有値の計算が終わらないな..
gpu_end = time.time()

print('実行時間：', gpu_end - gpu_start)
#print('eigen value:', w)
"""